## minTree
This code performs image segmentation using a max-tree algorithm to process a 3D greyscale image and an initial core segmentation.

Previous = (This file is a copy of the maxtree_new.py program.      New maxtree segmentation : considering a core segmentation, add it to the original image putting its value as a maximum value. The cores will correspond to leaves and the grains will be direct parents.)

## More information

- warning : Warning management
- argparse : To manage command line arguments.
- tifffile : For reading and writing TIFF files.
- numpy (alias np) : For manipulating numerical vectors.
- skimage : Image processing
- hg : An image processing library (a hierarchical processing library).

#### French

- warning : Gestion des avertissements
- argparse : Pour gérer les arguments de la ligne de commande.
- tifffile : Pour lire et écrire des fichiers TIFF.
- numpy (alias np) : Pour manipuler les tableaux numériques.
- skimage : Traitement d'images
- hg : Une bibliothèque de traitement d'images (une bibliothèque de traitement par hiérarchies).

# Code

In [ ]:
__author__ = "Lysandre Macke"
# Adapted by Jonathan Palisse
__contact__ = "lmacke@unistra.com"

### Imports

In [9]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    import higra as hg # ignore the generated warnings

import argparse
import numpy as np
import skimage.morphology as morph
from skimage.measure import label
import tifffile # tiff file manipulation

### Command-line program management

Command example : \
python maxTree.py image.tif noyau.tif 6\
python maxTree --help

In [ ]:
parser = argparse.ArgumentParser(
                    prog = "maxtree",
                    description = "test program.",
                    epilog = "For any question please contact me at lysandre.macke@unicaen.fr.")

parser.add_argument("file", help = "The path to the tiff file (shall be 3D grayscale)")
parser.add_argument("markers", help = "Path to cores segmentation (background shall be black)")
parser.add_argument("adjacency", help="Adjacency of the mask (6 or 26)")

args = parser.parse_args()
image_filepath = args.file
seed_filepath = args.markers
adjacency = args.adjacency

#### TIFF image loading and 8-bit conversion

In [ ]:
# image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
with tifffile.TiffFile(image_filepath) as tif:
    image = np.array([page.asarray() for page in tif.pages])

with tifffile.TiffFile(seed_filepath) as tif:
    cores = np.array([page.asarray() for page in tif.pages])

# convert to 8 bits
image = (image/256).astype("uint")
print("loaded image has shape :", image.shape)

#### Creating the result image and configuring the graph

neighbours is a formal representation of the connections between pixels \
https://higra.readthedocs.io/en/stable/python/graph_image.html

In [ ]:
if (adjacency == 26):
    # 26 adjacency implicit graph
    mask = [[[1, 1, 1], [1, 1, 1], [1, 1, 1]],
            [[1, 1, 1], [1, 0, 1], [1, 1, 1]],
            [[1, 1, 1], [1, 1, 1], [1, 1, 1]]]
else:
    # 6 adjacency implicit graph
    mask = [[[0, 0, 0], [0, 1, 0], [0, 0, 0]],
            [[0, 1, 0], [1, 0, 1], [0, 1, 0]],
            [[0, 0, 0], [0, 1, 0], [0, 0, 0]]]

neighbours = hg.mask_2_neighbours(mask)
graph = hg.get_nd_regular_implicit_graph(image.shape, neighbours)

#### Merging Image and Markers

In [ ]:
# overlap two images
cores = morph.dilation(cores, morph.ball(2.2))

components = label(cores, return_num=True)
total = components[1]
print("number of cores in the image :", total)

cores_val = image.max() + 1
image[cores > 0] = cores_val

tifffile.imwrite("tmp.tiff", image)

#### Building the maximal tree

In [ ]:
# construct max tree from image
tree, altitudes = hg.component_tree_max_tree(graph, image)

#### Calculating attributes and computing labels

In [ ]:
parents = tree.parents()
parents_size = len(parents)
labels = [0 for _ in range(parents_size)]
count  = [0 for _ in range(parents_size)]

depth = hg.attribute_depth(tree)
min_depth = 0.05*depth.max()

label_index = 0

print("start computing attributes")
# place attribute
for leaf in tree.leaves():
    # if we are in a new core region
    if(altitudes[leaf] == cores_val and count[tree.parent(leaf)] == 0):
        node = leaf
        while(node != tree.root()):
            count[node] += 1
            labels[node] = label_index
            node = tree.parent(node)
            
        label_index += 1

#### Node filtering

In [ ]:
print("filtering...")
for node in tree.leaves_to_root_iterator(include_leaves = False, include_root = True):
    if(count[node] > 1):
        labels[node] = 0

#### Image reconstruction and saving

In [ ]:
labels = np.asarray(labels)
res = hg.reconstruct_leaf_data(tree, labels)
tifffile.imwrite("res.tif", res)

#### Print information

In [ ]:
components = label(res, return_num=True)
total = components[1]
print("number of components in the image :", total)
print("number of labels :", max(labels))